In [1]:
import time

notebook_start_time = time.time()

In [2]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

root_dir = str(Path().absolute().parent)
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

Adding the following directory to the PYTHONPATH: /home/massyl/projects/personalized-recommender-project


In [3]:
import warnings

warnings.filterwarnings("ignore")

from loguru import logger

from recsys.hopsworks_integration import feature_store, two_tower_serving
from recsys.features import embeddings
from recsys.config import settings

2025-08-16 21:11:46.078105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-16 21:11:46.094200: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-16 21:11:46.099723: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-16 21:11:46.115270: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-16 21:11:46.982941: W tensorflow/compiler/tf2

# Offline inference pipeline: Computing item embeddings

In [4]:
project, fs = feature_store.get_feature_store()

mr = project.get_model_registry()

2025-08-16 21:11:48.456 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:10 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


2025-08-16 21:11:48,457 INFO: Initializing external client
2025-08-16 21:11:48,458 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-08-16 21:11:49,288 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/90261


# Computing candidate embeddings

In [5]:
candidate_model, candidate_features = (
    two_tower_serving.HopsworksCandidateModel.download(mr=mr)
)

2025-08-16 21:11:50.774 | INFO     | recsys.hopsworks_integration.two_tower_serving:download:115 - Downloading 'candidate_model' version 1
Downloading: 100.000%|██████████| 423/423 elapsed<00:00 remaining<00:00


Downloading: 100.000%|██████████| 767146/767146 elapsed<00:00 remaining<00:00


Downloading: 100.000%|██████████| 326323/326323 elapsed<00:00 remaining<00:00


Downloading: 100.000%|██████████| 56/56 elapsed<00:00 remaining<00:00
I0000 00:00:1755371513.861278   15024 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.


2025-08-16 21:11:53.955648: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Get candidates data

In [6]:
feature_view = fs.get_feature_view(
    name="retrieval",
    version=1,
)

In [7]:
train_df, val_df, test_df, _, _, _ = feature_view.train_validation_test_split(
    validation_size=settings.TWO_TOWER_DATASET_VALIDATON_SPLIT_SIZE,
    test_size=settings.TWO_TOWER_DATASET_TEST_SPLIT_SIZE,
    description="Retrieval dataset splits",
)
train_df.head(3)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.73s) 


,customer_id,article_id,t_dat,price,month_sin,month_cos,age,club_member_status,age_group,garment_group_name,index_group_name
0,f7048acb8188d98bde3a5c495475a3c86faafe0eede1f2...,670265002,0,0.013542,-8.660254e-01,0.500000,48.0,ACTIVE,46-55,"Under-, Nightwear",Ladieswear
1,5d34f84e6cbe9ec4706872bb65376097af1e53f0c7dac5...,751471035,0,0.033881,1.224647e-16,-1.000000,30.0,ACTIVE,26-35,Trousers,Ladieswear
2,baf6dc7ea8575732794751bb80824fe84fd40e6af86193...,719308002,0,0.059305,5.000000e-01,-0.866025,48.0,ACTIVE,46-55,Dresses Ladies,Divided


## Compute embeddings

In [8]:
item_df = embeddings.preprocess(train_df, candidate_features)
item_df.head(3)

,garment_group_name,article_id,index_group_name
0,"Under-, Nightwear",670265002,Ladieswear
1,Trousers,751471035,Ladieswear
2,Dresses Ladies,719308002,Divided


In [9]:
embeddings_df = embeddings.embed(df=item_df, candidate_model=candidate_model)
embeddings_df.head()

2025-08-16 21:12:09.268488: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-08-16 21:12:09.303938: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,article_id,embeddings
0,670265002,"[-0.6518390774726868, 1.3088957071304321, -0.7..."
1,751471035,"[1.4594303369522095, -0.5549032688140869, -0.5..."
2,719308002,"[1.7055261135101318, -0.3403882682323456, -0.5..."
3,759231002,"[1.2601754665374756, 0.02675279974937439, -1.1..."
4,793911001,"[1.9444431066513062, -0.18276336789131165, 0.3..."


# Create Hopsworks Embedding Index

In [10]:
candidate_embeddings_fg = (
    feature_store.create_candidate_embeddings_feature_group(
        fs=fs, df=embeddings_df, online_enabled=True
    )
)
logger.info("✅ Uploaded 'candidate_embeddings' Feature Group to Hopsworks")

Uploading Dataframe: 100.00% |██████████| Rows 11842/11842 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: candidate_embeddings_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/90261/jobs/named/candidate_embeddings_1_offline_fg_materialization/executions
2025-08-16 21:12:52,833 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-08-16 21:12:58,716 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-08-16 21:14:43,027 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-08-16 21:14:43,169 INFO: Waiting for log aggregation to finish.
2025-08-16 21:14:59,639 INFO: Execution finished successfully.


Online data ingestion progress: 23.44% |██▎       | Rows 2776/11842
2025-08-16 21:16:00.607 | INFO     | __main__:<module>:6 - ✅ Uploaded 'candidate_embeddings' Feature Group to Hopsworks


## Expose it to the online inference pipeline as a Feature View

In [11]:
feature_view = (
    feature_store.create_candidate_embeddings_feature_view(
        fs=fs, fg=candidate_embeddings_fg
    )
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/90261/fs/90180/fv/candidate_embeddings/version/1


In [12]:
notebook_end_time = time.time()
notebook_execution_time = notebook_end_time - notebook_start_time

logger.info(
    f"⌛️ Notebook Execution time: {notebook_execution_time:.2f} seconds ~ {notebook_execution_time / 60:.2f} minutes"
)

2025-08-16 21:16:26.847 | INFO     | __main__:<module>:4 - ⌛️ Notebook Execution time: 282.02 seconds ~ 4.70 minutes
